# Execution

In [ ]:
import json


def pr(r):
    if r.errors:
        errors = [
            {
                "message": e.message, "locations": [
                    {
                        "line": lo.line, "column": lo.column
                    } for lo in e.locations
                ] if e.locations else []
            } for e in r.errors]
        s = json.dumps(errors, indent=4).replace("\n", "\n    ")
        return f"Error: {s}"

    s = json.dumps(r.data, indent=4).replace("\n", "\n    ")
    return f"Data: {s}"

## 1. Executing a query

For executing a query against a schema, you can directly call the `execute` method on it.

In [ ]:
from graphene import Schema


schema = Schema(query=..., mutation=...)
variables = {...}
result = schema.execute("{ ... }", variables=variables)

`result` represents the result of execution, `result.data` is the result of execcuting he query, `result.errors` is `None` if no errors occurred, and is a non-empty list if an error occurred.

### 1.1. Context

You can pass context to a query via `context`.

In [ ]:
from graphene import ObjectType, String, Schema


class Query(ObjectType):
    name = String()

    def resolve_name(self, info):
        return info.context.get("name")


schema = Schema(query=Query)

context = {
    "name": "Alvin"
}
q = """
    {
        name
    }"""
r = schema.execute(q, context=context)
print(f'* The "r.data" of query "{q}, context={context}\n" is: "{pr(r)}"')

### 1.2. Variables

You can pass variables to a query via `variables`.

In [ ]:
from graphene import ObjectType, Field, ID, String, Schema


class User(ObjectType):
    id = ID(required=True)
    name = String(required=True)


dataset = {
    1: User(id=1, name="Alvin"),
    2: User(id=2, name="Emma"),
    3: User(id=3, name="Lucy")
}


class Query(ObjectType):
    user = Field(User, id=ID(required=True))

    def resolve_user(self, info, id):
        return dataset[int(id)]


schema = Schema(query=Query)

q = """
    query($id: ID!) {
        user(id: $id) {
            id
            name
        }
    }"""
v = {"id": 2}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}, variables={v}\n" is: "{pr(r)}"')

### 1.3. Root Value

Value used for **Parent Value Object (parent)** in root queries and mutations can be overridden using `root` parameter

In [ ]:
from graphene import ObjectType, ID, String, Field, Schema


class User(ObjectType):
    id = ID(required=True)
    name = String(required=True)

    def __str__(self):
        return str({"id": self.id, "name": self.name})


dataset = {
    1: User(id=1, name="Alvin"),
    2: User(id=2, name="Emma"),
    3: User(id=3, name="Lucy")
}


class Query(ObjectType):
    user = Field(User, id=ID())

    def resolve_user(self, info, id: str = None):  # The "self" is "root" variables pass from "execute"
        return self if self else dataset[int(id)]


schema = Schema(query=Query)

q = """
    query($id: ID!) {
        user(id: $id) {
            id
            name
        }
    }"""
root = None
v = {"id": 2}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}, root={root}, variables={v}\n" '
      f'is: "{pr(r)}"')

q = """
    query {
        user {
            id
            name
        }
    }"""
root = User(id=1, name="Alvin")
v = None
r = schema.execute(q, root=root)
print(f'* The "r.data" of query "{q}, root={root}, variables={v}\n" '
      f'is: "{pr(r)}"')

### 1.4. Operation Name

If there are multiple operations defined in a query string, `operation_name` should be used to indicate which should be executed.

In [ ]:
from graphene import ObjectType, Field, ID, Schema


class User(ObjectType):
    id = ID(required=True)
    first_name = String(required=True)
    last_name = String(required=True)
    full_name = String(required=True)

    def resolve_full_name(self, info):
        return f"{self.first_name} {self.last_name}"


dataset = {
    1: User(id=1, first_name="Alvin", last_name="Qu"),
    2: User(id=2, first_name="Emma", last_name="Yua"),
    3: User(id=3, first_name="Lucy", last_name="Green")
}


class Query(ObjectType):
    user = Field(User, id=ID(required=True))

    def resolve_user(self, info, id: str):
        return dataset[int(id)]


schema = Schema(query=Query)

q = """
    query getUserWithFirstName($id: ID!) {
        user(id: $id) {
            id
            firstName
            lastName
        }
    }

    query getUserWithFullName($id: ID!) {
        user(id: $id) {
            id
            fullName
        }
    }"""

opt_name = "getUserWithFirstName"
v = {"id": 3}
r = schema.execute(q, operation_name=opt_name, variables=v)
print(f'* The "r.data" of query "{q}, '
      f'operation_name="{opt_name}", '
      f'variables={v}\n" is: "{pr(r)}"')

opt_name = "getUserWithFullName"
v = {"id": 2}
r = schema.execute(q, operation_name=opt_name, variables=v)
print(f'* The "r.data" of query "{q}, '
      f'operation_name="{opt_name}", '
      f'variables={v}\n" is: "{pr(r)}"')

## 2. Middleware

You can use `middleware` to affect the evaluation of fields in your schema.

A middleware is any object or function that responds to `resolve(next_middleware, *args)`.

Iside that method, it should either:

- Send `resolve` to the next middleware to continue the evaluation; or
- Return a value to end the evaluation early.

### 2.1. Resolve arguments

Middlewares `resolve` is invoked with several arguments:

- `next` represents the execution chain. Call `next` to continue evaluation.
- `instance` is the instance value object passed throughtout the query.
- `info` is the resolver info.
- `args` is the dict of arguments passed to the field.

### 2.2. Example

In [ ]:
from graphene import (
    ObjectType,
    InputObjectType,
    ID,
    String,
    Int,
    Field,
    Argument,
    Mutation,
    Schema
)


class User(ObjectType):
    id = ID(required=True)
    name = String(required=True)
    age = Int()
    nickname = String()

    def __str__(self):
        return f'User(id={self.id}, name="{self.name}", age={self.age}, nickname="{self.nickname}")'


class UserInput(InputObjectType):
    name = String(required=True)
    age = Int()


class UserCreate(Mutation):
    class Arguments:
        user_input = UserInput(required=True)

    Output = User

    def mutate(self, info, user_input):
        return User(id=100, name=user_input.name, age=user_input.age)


class Query(ObjectType):
    user = Field(User, id=Argument(ID, required=True))

    def resolve_user(self, info, id: str):
        id = int(id)
        if id == 1:
            return User(id=1, name="Alvin", age=40)

        return User(id=10, name="Emma", age=36)


class UserMutation(ObjectType):
    user_create = UserCreate.Field()


schema = Schema(query=Query, mutation=UserMutation)

This middleware does two things:
- Return value *"purpleswg"* when field name is **"nickname"** when `user.id` equals 1, to make sure the instance object `user` has field `nickname` with value "purpleswg"
- Change the "age" attribute value of the "user_input" argument to `1024` if the "name" attribute is "Authur"

In [ ]:
# Define a "Middleware Class"
class AuthorizationMiddleware:
    @staticmethod
    def pretty(s):
        return str(s).replace("\n", "\n        ")

    def resolve(self, next, instance, info, **kwargs):
        print(f'  >> resolve the field="{info.field_name}" '
              f'with instance={instance} and args={kwargs}')

        # Check is "mutation" operation with name "createUser"
        if (info.operation.operation == "mutation" and
                info.operation.name.value == "createUser"):

            # check if "Mutation" field is "user_create"
            if info.field_name == "userCreate":
                # Check if input name and modify the input age argument
                user_input = kwargs["user_input"]
                if user_input.name == "Authur":
                    user_input.age = 1024

        next_node = next(instance, info, **kwargs)

#         Show the "instance", "info" and "kwargs" arguments:
#         print(f'  >> the "instance" arguments is: {instance}')
#         print(f'  >> the "info" arguments is: \n'
#               f'  >>>> "info.context" is: {info.context}\n'
#               f'  >>>> "info.field_asts" is: {info.field_asts}\n'
#               f'  >>>> "info.field_name" is: {info.field_name}\n'
#               f'  >>>> "info.fragments" is: {info.fragments}\n'
#               f'  >>>> "info.operation" is: {info.operation}\n'
#               f'  >>>> "info.parent_type" is: {info.parent_type}\n'
#               f'  >>>> "info.path" is: {info.path}\n'
#               f'  >>>> "info.return_type" is: {info.return_type}\n'
#               f'  >>>> "info.root_value" is: {info.root_value}\n'
#               f'  >>>> "info.schema" is: {self.pretty(info.schema)}\n'
#               f'  >>>> "info.variable_values" is: {info.variable_values}')

        # Check is "query" operation with name "getUserById"
        if (info.operation.operation == "query" and
                info.operation.name.value == "getUserById"):

            # if resolve is "get nickname", and "user.id" is 1, return nickname
            if info.field_name == "nickname":
                if instance.id == 1:
                    return "purpleswg"

        return next_node

And then execute it with:

In [ ]:
q = """
    query getUserById($id: ID!) {
        user(id: $id) {
            id
            name
            age
            nickname
        }
    }

    mutation createUser($user: UserInput!) {
        userCreate(userInput: $user) {
            id
            name
            age
        }
    }"""

opt_name = "getUserById"
v = {"id": 1}
r = schema.execute(
    q,
    operation_name=opt_name,
    variables=v,
    middleware=[AuthorizationMiddleware()]
)
print(f'* The "r.data" of query "{q}, '
      f'operation_name="{opt_name}", '
      f'variables={v}\n" is: "{pr(r)}"')

opt_name = "createUser"
v = {"user": {"name": "Authur", "age": 42}}
r = schema.execute(
    q,
    operation_name=opt_name,
    variables=v,
    middleware=[AuthorizationMiddleware()]
)
print(f'* The "r.data" of query "{q}, '
      f'operation_name="{opt_name}", '
      f'variables={v}\n" is: "{pr(r)}"')

### 2.3. Functional example

Middleware can also be defined as a function. Here we define a middleware that logs the time it takes to resolve each field

In [ ]:
from time import time as timer


# Define a "Middleware function"
def timing_middleware(next, instance, info, **kwargs):
    start = timer()
    next_node = next(instance, info, **kwargs)

    duration = round((timer() - start) * 1000, 2)
    parent_type_name = instance._meta.name if instance and hasattr(instance, "_meta") else ""
    print(f"  >> {parent_type_name}.{info.field_name}: {duration} ms")

    return next_node

And then execute it with:

In [ ]:
q = """
    query getUserById($id: ID!) {
        user(id: $id) {
            id
            name
            age
            nickname
        }
    }

    mutation createUser($user: UserInput!) {
        userCreate(userInput: $user) {
            id
            name
            age
        }
    }"""

opt_name = "getUserById"
v = {"id": 1}
r = schema.execute(
    q,
    operation_name=opt_name,
    variables=v,
    middleware=[timing_middleware]
)
print(f'* The "r.data" of query "{q}, '
      f'operation_name="{opt_name}", '
      f'variables={v}\n" is: "{pr(r)}"')

opt_name = "createUser"
v = {"user": {"name": "Authur", "age": 42}}
r = schema.execute(
    q,
    operation_name=opt_name,
    variables=v,
    middleware=[timing_middleware]
)
print(f'* The "r.data" of query "{q}, '
      f'operation_name="{opt_name}", '
      f'variables={v}\n" is: "{pr(r)}"')

## 3. Dataloader

Dataloader is a generic utility to be used as part of your application's data fetching layer to provide a simplified and consistent API over various remote data sources such as databases or web services via batching and caching.

### 3.1. Batching

Batching is not an advanced feature, it's *Dataloader's* primary feature. Create loaders by providing a batch loading function.

In [ ]:
from promise import Promise
from promise.dataloader import DataLoader


def get_user(id):
    ...


class UserLoader(DataLoader):
    def batch_load_fn(self, keys):
        # Here we return a promise that will result on the
        # corresponding user for each key in keys
        return Promise.resolve([get_user(id=key) for key in keys])

A batch loading function accepts a list of keys, and return a `Promise` which resoves to a list of `value`.

Then load individual values from the loader. `Dataloader` will coalesce all individual loads which occur within a single frame of execution (executed once the wrapping promise is resolved) and then call your batch function with all requested keys.

In [ ]:
user_loader = UserLoader()

user_loader.load(1).then(lambda user: user_loader.load(user.best_friend_id))
user_loader.load(2).then(lambda user: user_loader.load(user.best_friend_id))

`Dataloader` allows you to decouple unrelated parts of your application without sacrificing the performance of batch data-loading. While the loader presents an API tht loads individual values, all concurrent requests will be coalesced and presented to your batch loading function. This allows your application to safely distribute data fetching requirements throughtout your application and maintain minimal outgoing data requrests.

### 3.2. Using with Graphene

Dataloader pairs nicely well with Graphene/GraphQL. GraphQL fields are designed to be stand-alone functions. Without a caching or batching mechanism. it's easy for a naive GraphQL server to issuse new database requests each time a field is resolved. 

Consider the floolwing GraphQL request:

```graphql
{
  me {
    name
    bestFriend {
      name
    }
    friends(first: 5) {
      name
      bestFriend {
        name
      }
    }
  }
}
```

Naively, if `me` `bestfriend` and `friends` each need to request the backend, there could be at most 13 database requests!

When using *Dataloader*, we could define the User type using out previous example with leaner code and at most 4 database requests, and possibly fewer if there are cache hits.

Examples:

- Define dataset and Dataloader

In [ ]:
from promise import Promise
from promise.dataloader import DataLoader


class Dataset:
    def __init__(self):
        self.users = {}

    def save_user(self, user):
        self.users[user.id] = user

    def get_user(self, id):
        return self.users[id]


dataset = Dataset()


class UserLoader(DataLoader):
    def batch_load_fn(self, keys):
        users = map(lambda key: dataset.get_user(key), keys)
        return Promise.resolve(list(users))


user_loader = UserLoader()

- Define ObjectTypes and Schema

In [ ]:
import random
from graphene import ObjectType, ID, String, Int, List, Field, Argument, Schema


class User(ObjectType):
    id = ID(required=True)
    name = String(required=True)
    friends = List(lambda: User)
    best_friend = Field(lambda: User)

    def resolve_friends(self, info):
        return user_loader.load_many(self.friends)

    def resolve_best_friend(self, info):
        return user_loader.load(self.best_friend)


class Query(ObjectType):
    user = Field(User, id=Argument(ID, required=True))

    def resolve_user(self, info, id):
        return user_loader.load(int(id))


schema = Schema(query=Query)

- Fill data to dataset

In [ ]:
import random


for id in range(1, 101):
    friend_ids = [n for n in random.sample(range(0, 101), 4) if n != id]
    best_friend_id = friend_ids[random.randint(0, len(friend_ids) - 1)]

    dataset.save_user(
        User(
            id=id,
            name=f"user-{id}",
            friends=friend_ids,
            best_friend=best_friend_id
        )
    )

- Execute query

In [ ]:
q = """
    query getUser($id: ID!) {
        user(id: $id) {
            id
            name
            friends {
                __typename
                id
                name
            }
            bestFriend {
                __typename
                id
                name
            }
        }
    }"""

v = {"id": random.randint(1, 100)}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}, variables={v}\n" is: "{pr(r)}"')

## 4. File uploading

File uploading is not part of the official GraphQL spec yet and is not natively implemented in Graphene.

If your server needs to support file uploading then you can use the libary: [graphene-file-upload](https://github.com/lmcgartland/graphene-file-upload) which enhances Graphene to add file uploads and conforms to the unoffical GraphQL [multipart request spec](https://github.com/jaydenseric/graphql-multipart-request-spec).

### 4.1. GraphQL multipart request specification

An interoperable multipart form field structure for GraphQL requests, used by various file upload client/server implementations.

It’s possible to implement:

- Nesting files anywhere within operations (usually in variables).
- Operation batching.
- File deduplication.
- File upload streams in resolvers.
- Aborting file uploads in resolvers.

![](./assets/sync-vs-async-graphql-multipart-request-middleware.svg)

#### 4.1.1. Multipart form field structure

An "operations object" is an [Apollo GraphQL POST request](https://www.apollographql.com/docs/apollo-server/requests/#postRequests) (or array of requests if batching). An "operations path" is an [`object-path`](https://www.npmjs.com/package/object-path) string to locate a file within an operations object.

So operations can be resolved while the files are still uploading, the fields are ordered:

1. `operations`: A JSON encoded operations object with files replaced with `null`.
2. `map`: A JSON encoded map of where files occurred in the operations. For each file, the key is the file multipart form field name and the value is an array of operations paths.
3. File fields: Each file extracted from the operations object with a unique, arbitrary field name.

### 4.2. Example

#### 4.2.1. Single file

**Operations**


```javascript
{
  query: `
    mutation($file: Upload!) {
      singleUpload(file: $file) {
        id
      }
    }
  `,
  variables: {
    file: File // a.txt
  }
}
```

**cURL request**

```bash
$ curl localhost:3001/graphql \
  -F operations='{ "query": "mutation ($file: Upload!) { singleUpload(file: $file) { id } }", "variables": { "file": null } }' \
  -F map='{ "0": ["variables.file"] }' \
  -F 0=@a.txt
```

**Request payload**

```plain
--------------------------cec8e8123c05ba25
Content-Disposition: form-data; name="operations"

{ "query": "mutation ($file: Upload!) { singleUpload(file: $file) { id } }", "variables": { "file": null } }
--------------------------cec8e8123c05ba25
Content-Disposition: form-data; name="map"

{ "0": ["variables.file"] }
--------------------------cec8e8123c05ba25
Content-Disposition: form-data; name="0"; filename="a.txt"
Content-Type: text/plain

Alpha file content.

--------------------------cec8e8123c05ba25--
```

#### 4.2.2. File list

**Operations**

```javascript
{
  query: `
    mutation($files: [Upload!]!) {
      multipleUpload(files: $files) {
        id
      }
    }
  `,
  variables: {
    files: [
      File, // b.txt
      File // c.txt
    ]
  }
}
```

**cURL request**

```bash
$ curl localhost:3001/graphql \
  -F operations='{ "query": "mutation($files: [Upload!]!) { multipleUpload(files: $files) { id } }", "variables": { "files": [null, null] } }' \
  -F map='{ "0": ["variables.files.0"], "1": ["variables.files.1"] }' \
  -F 0=@b.txt \
  -F 1=@c.txt
```


**Request payload**

```plain
--------------------------ec62457de6331cad
Content-Disposition: form-data; name="operations"

{ "query": "mutation($files: [Upload!]!) { multipleUpload(files: $files) { id } }", "variables": { "files": [null, null] } }
--------------------------ec62457de6331cad
Content-Disposition: form-data; name="map"

{ "0": ["variables.files.0"], "1": ["variables.files.1"] }
--------------------------ec62457de6331cad
Content-Disposition: form-data; name="0"; filename="b.txt"
Content-Type: text/plain

Bravo file content.

--------------------------ec62457de6331cad
Content-Disposition: form-data; name="1"; filename="c.txt"
Content-Type: text/plain

Charlie file content.

--------------------------ec62457de6331cad--
```

#### 4.2.3. Batching

**Operations**

```javascript
;[
  {
    query: `
      mutation($file: Upload!) {
        singleUpload(file: $file) {
          id
        }
      }
    `,
    variables: {
      file: File // a.txt
    }
  },
  {
    query: `
      mutation($files: [Upload!]!) {
        multipleUpload(files: $files) {
          id
        }
      }
    `,
    variables: {
      files: [
        File, // b.txt
        File // c.txt
      ]
    }
  }
]
```

**cURL request**

```bash
$ curl localhost:3001/graphql \
  -F operations='[{ "query": "mutation ($file: Upload!) { singleUpload(file: $file) { id } }", "variables": { "file": null } }, { "query": "mutation($files: [Upload!]!) { multipleUpload(files: $files) { id } }", "variables": { "files": [null, null] } }]' \
  -F map='{ "0": ["0.variables.file"], "1": ["1.variables.files.0"], "2": ["1.variables.files.1"] }' \
  -F 0=@a.txt \
  -F 1=@b.txt \
  -F 2=@c.txt
```


**Request payload**

```plain
--------------------------627436eaefdbc285
Content-Disposition: form-data; name="operations"

[{ "query": "mutation ($file: Upload!) { singleUpload(file: $file) { id } }", "variables": { "file": null } }, { "query": "mutation($files: [Upload!]!) { multipleUpload(files: $files) { id } }", "variables": { "files": [null, null] } }]
--------------------------627436eaefdbc285
Content-Disposition: form-data; name="map"

{ "0": ["0.variables.file"], "1": ["1.variables.files.0"], "2": ["1.variables.files.1"] }
--------------------------627436eaefdbc285
Content-Disposition: form-data; name="0"; filename="a.txt"
Content-Type: text/plain

Alpha file content.

--------------------------627436eaefdbc285
Content-Disposition: form-data; name="1"; filename="b.txt"
Content-Type: text/plain

Bravo file content.

--------------------------627436eaefdbc285
Content-Disposition: form-data; name="2"; filename="c.txt"
Content-Type: text/plain

Charlie file content.

--------------------------627436eaefdbc285--
```

## 5. Subscriptions

To create a subscription, you can directly call the `subscribe` method on the schema. This method is async and must be awaited.

In [ ]:
import asyncio
from datetime import datetime, timedelta
from graphene import ObjectType, String, Schema, Field


# Every schema requires a query.
class Query(ObjectType):
    hello = String()

    def resolve_hello(self, info):
        ...


class Subscription(ObjectType):
    time_of_day = String()

    async def subscribe_time_of_day(self, info):
        start = datetime.utcnow()
        while (datetime.utcnow() - start) < timedelta(seconds=5):
            yield datetime.utcnow().isoformat()
            await asyncio.sleep(1)


async def run(schema):
    subscription = "subscription { timeOfDay }"
    rs = schema.subscribe(subscription)
    async for item in await rs:
        print(f'  the "item.data" of subscription "{subscription}" is: "{pr(item)}"')


schema = Schema(query=Query, subscription=Subscription)
await run(schema)

Note: **Subscriptions** feature is valid in V3